<a href="https://colab.research.google.com/github/siddugoud6966/NLP_2024-2025/blob/main/NLP_ASS_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use a small dataset of English to French sentence pairs. You can replace this with any other language pair dataset. data = [ ("hello", "bonjour"), ("how are you", "comment ça va"), ("I am fine", "je vais bien"), ("what is your name", "comment tu t'appelles"), ("my name is", "je m'appelle"), ("thank you", "merci"), ("goodbye", "au revoir") ] [CO5] (a) Take input (English) and target (French)

In [6]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]

input_texts = [pair[0] for pair in data]
target_texts = ['\t' + pair[1] + '\n' for pair in data]

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    print(f"Example {i+1}:")
    print(f"Input: {input_text}")
    print(f"Target: {target_text}")
    print()


Example 1:
Input: hello
Target: 	bonjour


Example 2:
Input: how are you
Target: 	comment ça va


Example 3:
Input: I am fine
Target: 	je vais bien


Example 4:
Input: what is your name
Target: 	comment tu t'appelles


Example 5:
Input: my name is
Target: 	je m'appelle


Example 6:
Input: thank you
Target: 	merci


Example 7:
Input: goodbye
Target: 	au revoir




(b) Building the Seq2Seq Model

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]

input_texts = [pair[0] for pair in data]
target_texts = ['\t' + pair[1] + '\n' for pair in data]

input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts)
input_seq = input_tokenizer.texts_to_sequences(input_texts)

target_tokenizer = Tokenizer()
target_tokenizer.fit_on_texts(target_texts)
target_seq = target_tokenizer.texts_to_sequences(target_texts)

input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

max_input_length = max(len(seq) for seq in input_seq)
max_target_length = max(len(seq) for seq in target_seq)

input_seq = pad_sequences(input_seq, maxlen=max_input_length, padding='post')
target_seq = pad_sequences(target_seq, maxlen=max_target_length, padding='post')

decoder_input_seq = target_seq[:, :-1]
decoder_output_seq = target_seq[:, 1:]

decoder_output_seq_onehot = np.zeros((len(decoder_output_seq), max_target_length - 1, target_vocab_size), dtype="float32")
for i, seq in enumerate(decoder_output_seq):
    for t, word in enumerate(seq):
        decoder_output_seq_onehot[i, t, word - 1] = 1.0

encoder_inputs = Input(shape=(max_input_length,))
encoder_embedding = Embedding(input_vocab_size, 256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_target_length - 1,))
decoder_embedding = Embedding(target_vocab_size, 256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_lstm_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.fit([input_seq, decoder_input_seq], decoder_output_seq_onehot, epochs=10, batch_size=2, validation_split=0.2)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 4)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_5             │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 4, 256)         │          3,840 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 2, 256)         │          3,584 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ [(None, 256), (None,   │        525,312 │ embedding[0][0]        │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ [(None, 2, 256),       │        525,312 │ embedding_1[0][0],     │
│                           │ (None, 256), (None,    │                │ lstm_2[0][1],          │
│                           │ 256)]                  │                │ lstm_2[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 2, 14)          │          3,598 │ lstm_3[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,061,646 (4.05 MB)

 Trainable params: 1,061,646 (4.05 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 251ms/step - accuracy: 0.0000e+00 - loss: 2.6350 - val_accuracy: 0.7500 - val_loss: 2.5758
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.4062 - loss: 2.5483 - val_accuracy: 0.7500 - val_loss: 2.4998
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.4688 - loss: 2.4458 - val_accuracy: 0.7500 - val_loss: 2.3722
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.5000 - loss: 2.3550 - val_accuracy: 0.7500 - val_loss: 2.1931
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.5312 - loss: 2.1650 - val_accuracy: 0.7500 - val_loss: 1.8936
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.5312 - loss: 1.9260 - val_accuracy: 0.7500 - val_loss: 1.5707
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.3875 - loss: 1.6879 - val_accuracy: 0.7500 - val_loss: 1.5261
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.3688 - loss: 1.3301 - val_accuracy: 0.7500 - val_loss: 1.

In [9]:
# Train the Seq2Seq model
model.fit([input_seq, decoder_input_seq], decoder_output_seq_onehot, epochs=10, batch_size=2, validation_split=0.2)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.8562 - loss: 0.9475 - val_accuracy: 0.7500 - val_loss: 2.3946
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 1.0000 - loss: 0.5889 - val_accuracy: 0.7500 - val_loss: 2.6977
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 1.0000 - loss: 0.3981 - val_accuracy: 0.7500 - val_loss: 3.1835
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 1.0000 - loss: 0.3480 - val_accuracy: 0.7500 - val_loss: 3.5641
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 0.2538 - val_accuracy: 0.7500 - val_loss: 3.7548
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 1.0000 - loss: 0.2098 - val_accuracy: 0.7500 - val_loss: 3.8700
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 1.0000 - loss: 0.1599 - val_accuracy: 0.7500 - val_loss: 4.0401
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 1.0000 - loss: 0.0891 - val_accuracy: 0.7500 - val_loss: 4.2423


(d) After training, set up separate models for the encoder and decoder to perform inference (translation) on new sentences.

In [10]:
# Decoder's input for inference
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Decoder LSTM with stateful behavior
decoder_lstm_output, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

# Dense layer for prediction
decoder_output = decoder_dense(decoder_lstm_output)

# Define decoder model
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_output] + decoder_states)

(e) Use the trained model to translate new English sentences into French

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]

input_texts = [pair[0] for pair in data]
target_texts = ['\t' + pair[1] + '\n' for pair in data]

input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts)
input_seq = input_tokenizer.texts_to_sequences(input_texts)

target_tokenizer = Tokenizer()
target_tokenizer.fit_on_texts(target_texts)
target_seq = target_tokenizer.texts_to_sequences(target_texts)

target_tokenizer.word_index['\t'] = len(target_tokenizer.word_index) + 1
target_tokenizer.word_index['\n'] = len(target_tokenizer.word_index) + 1

target_tokenizer.index_word[target_tokenizer.word_index['\t']] = '\t'
target_tokenizer.index_word[target_tokenizer.word_index['\n']] = '\n'

input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

max_input_length = max(len(seq) for seq in input_seq)
max_target_length = max(len(seq) for seq in target_seq)

input_seq = pad_sequences(input_seq, maxlen=max_input_length, padding='post')
target_seq = pad_sequences(target_seq, maxlen=max_target_length, padding='post')

decoder_input_seq = target_seq[:, :-1]
decoder_output_seq = target_seq[:, 1:]

decoder_output_seq_onehot = np.zeros((len(decoder_output_seq), max_target_length - 1, target_vocab_size), dtype="float32")
for i, seq in enumerate(decoder_output_seq):
    for t, word in enumerate(seq):
        decoder_output_seq_onehot[i, t, word - 1] = 1.0

encoder_inputs = Input(shape=(max_input_length,))
encoder_embedding = Embedding(input_vocab_size, 256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_target_length-1,))
decoder_embedding = Embedding(target_vocab_size, 256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_lstm_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model.fit([input_seq, decoder_input_seq], decoder_output_seq_onehot, epochs=10, batch_size=2, validation_split=0.2, callbacks=[early_stopping])

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_output, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

decoder_output = decoder_dense(decoder_lstm_output)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_output] + decoder_states)

def translate(input_text):

    input_seq = input_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_input_length, padding='post')

    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_tokenizer.word_index['\t']

    translated_sentence = ''

    for _ in range(max_target_length - 1):

        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = target_tokenizer.index_word[sampled_token_index]

        if sampled_word == '\n':
            break

        translated_sentence += ' ' + sampled_word

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return translated_sentence.strip()

for input_sentence, target_sentence in zip(input_texts, target_texts):
    translated_sentence = translate(input_sentence)
    print(f"Input (English): {input_sentence}")
    print(f"Actual Translation (French): {target_sentence.strip()}")
    print(f"Translated Sentence (French): {translated_sentence}")
    print('-' * 40)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8             │ (None, 4)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_9             │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 4, 256)         │          3,840 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 2, 256)         │          4,096 │ input_layer_9[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_4 (LSTM)             │ [(None, 256), (None,   │        525,312 │ embedding_2[0][0]      │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_5 (LSTM)             │ [(None, 2, 256),       │        525,312 │ embedding_3[0][0],     │
│                           │ (None, 256), (None,    │                │ lstm_4[0][1],          │
│                           │ 256)]                  │                │ lstm_4[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 2, 16)          │          4,112 │ lstm_5[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,062,672 (4.05 MB)

 Trainable params: 1,062,672 (4.05 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 248ms/step - accuracy: 0.0000e+00 - loss: 2.7732 - val_accuracy: 0.7500 - val_loss: 2.7286
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.6313 - loss: 2.6895 - val_accuracy: 0.7500 - val_loss: 2.6370
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.7250 - loss: 2.5969 - val_accuracy: 0.7500 - val_loss: 2.4912
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5938 - loss: 2.4605 - val_accuracy: 0.7500 - val_loss: 2.2562
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2625 - loss: 2.3385 - val_accuracy: 0.7500 - val_loss: 1.9518
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.4313 - loss: 1.8557 - val_accuracy: 0.7500 - val_loss: 1.6218
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3063 - loss: 1.7218 - val_accuracy: 0.7500 - val_loss: 1.7819
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3688 - loss: 1.4076 - val_accuracy: 0.7500 - val_loss: 2.